# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [11]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [12]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "../Weather_Database/WeatherPy_database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv(file_path)

# Display sample data
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,70.11,51,77,11.88,broken clouds
1,1,Taft,IR,31.7471,54.2088,78.75,10,0,2.30,clear sky
2,2,San Patricio,US,28.0170,-97.5169,78.84,56,0,6.93,clear sky
3,3,Mahebourg,MU,-20.4081,57.7000,76.24,68,75,9.22,broken clouds
4,4,Saint George,US,37.1041,-113.5841,78.26,36,0,8.05,clear sky
5,5,Khatanga,RU,71.9667,102.5000,31.23,96,100,14.47,light snow
6,6,Mount Isa,AU,-20.7333,139.5000,83.97,17,99,4.61,overcast clouds
7,7,Bredasdorp,ZA,-34.5322,20.0403,58.05,70,100,2.33,overcast clouds
8,8,Mataura,NZ,-46.1927,168.8643,51.48,56,0,5.08,clear sky
9,9,Verkhoturye,RU,58.8627,60.8054,37.58,51,61,7.96,broken clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [13]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [14]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
                                    
# Display sample data
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,70.11,51,77,11.88,broken clouds
1,1,Taft,IR,31.7471,54.2088,78.75,10,0,2.30,clear sky
2,2,San Patricio,US,28.0170,-97.5169,78.84,56,0,6.93,clear sky
3,3,Mahebourg,MU,-20.4081,57.7000,76.24,68,75,9.22,broken clouds
4,4,Saint George,US,37.1041,-113.5841,78.26,36,0,8.05,clear sky
6,6,Mount Isa,AU,-20.7333,139.5000,83.97,17,99,4.61,overcast clouds
10,10,Faanui,PF,-16.4833,-151.7500,78.01,67,19,4.50,few clouds
12,12,Dabhoi,IN,22.1833,73.4333,87.73,54,1,6.31,clear sky
13,13,Kavieng,PG,-2.5744,150.7967,82.65,73,82,12.33,broken clouds
16,16,Yinchuan,CN,38.4681,106.2731,81.45,35,19,3.94,few clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [15]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,70.11,51,77,11.88,broken clouds
1,1,Taft,IR,31.7471,54.2088,78.75,10,0,2.30,clear sky
2,2,San Patricio,US,28.0170,-97.5169,78.84,56,0,6.93,clear sky
3,3,Mahebourg,MU,-20.4081,57.7000,76.24,68,75,9.22,broken clouds
4,4,Saint George,US,37.1041,-113.5841,78.26,36,0,8.05,clear sky
6,6,Mount Isa,AU,-20.7333,139.5000,83.97,17,99,4.61,overcast clouds
10,10,Faanui,PF,-16.4833,-151.7500,78.01,67,19,4.50,few clouds
12,12,Dabhoi,IN,22.1833,73.4333,87.73,54,1,6.31,clear sky
13,13,Kavieng,PG,-2.5744,150.7967,82.65,73,82,12.33,broken clouds
16,16,Yinchuan,CN,38.4681,106.2731,81.45,35,19,3.94,few clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [16]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp","Current Description", "Lat", "Lng"]].copy()

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng
0,Busselton,AU,70.11,broken clouds,-33.6500,115.3333
1,Taft,IR,78.75,clear sky,31.7471,54.2088
2,San Patricio,US,78.84,clear sky,28.0170,-97.5169
3,Mahebourg,MU,76.24,broken clouds,-20.4081,57.7000
4,Saint George,US,78.26,clear sky,37.1041,-113.5841
6,Mount Isa,AU,83.97,overcast clouds,-20.7333,139.5000
10,Faanui,PF,78.01,few clouds,-16.4833,-151.7500
12,Dabhoi,IN,87.73,clear sky,22.1833,73.4333
13,Kavieng,PG,82.65,broken clouds,-2.5744,150.7967
16,Yinchuan,CN,81.45,few clouds,38.4681,106.2731


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [17]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,70.11,broken clouds,-33.6500,115.3333,
1,Taft,IR,78.75,clear sky,31.7471,54.2088,
2,San Patricio,US,78.84,clear sky,28.0170,-97.5169,
3,Mahebourg,MU,76.24,broken clouds,-20.4081,57.7000,
4,Saint George,US,78.26,clear sky,37.1041,-113.5841,
6,Mount Isa,AU,83.97,overcast clouds,-20.7333,139.5000,
10,Faanui,PF,78.01,few clouds,-16.4833,-151.7500,
12,Dabhoi,IN,87.73,clear sky,22.1833,73.4333,
13,Kavieng,PG,82.65,broken clouds,-2.5744,150.7967,
16,Yinchuan,CN,81.45,few clouds,38.4681,106.2731,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [19]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [20]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [21]:
# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,70.11,broken clouds,-33.6500,115.3333,Observatory Guest House
1,Taft,IR,78.75,clear sky,31.7471,54.2088,عمارت ساعتی تفت
2,San Patricio,US,78.84,clear sky,28.0170,-97.5169,"Motel 6 Sinton, TX"
3,Mahebourg,MU,76.24,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
4,Saint George,US,78.26,clear sky,37.1041,-113.5841,Best Western Plus Abbey Inn
6,Mount Isa,AU,83.97,overcast clouds,-20.7333,139.5000,ibis Styles Mt Isa Verona
10,Faanui,PF,78.01,few clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
12,Dabhoi,IN,87.73,clear sky,22.1833,73.4333,Sindhiyapura
13,Kavieng,PG,82.65,broken clouds,-2.5744,150.7967,Nusa Island Retreat
16,Yinchuan,CN,81.45,few clouds,38.4681,106.2731,JJ Inns - Yinchuan New Moon Square Branch


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [36]:
# Drop the rows where there is no Hotel Name.
no_name_condition = hotel_df["Hotel Name"] == ""
index_names = hotel_df[no_name_condition].index
clean_hotel_df = hotel_df.drop(index_names)

# Display sample data
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,70.11,broken clouds,-33.6500,115.3333,Observatory Guest House
1,Taft,IR,78.75,clear sky,31.7471,54.2088,عمارت ساعتی تفت
2,San Patricio,US,78.84,clear sky,28.0170,-97.5169,"Motel 6 Sinton, TX"
3,Mahebourg,MU,76.24,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
4,Saint George,US,78.26,clear sky,37.1041,-113.5841,Best Western Plus Abbey Inn
6,Mount Isa,AU,83.97,overcast clouds,-20.7333,139.5000,ibis Styles Mt Isa Verona
10,Faanui,PF,78.01,few clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
12,Dabhoi,IN,87.73,clear sky,22.1833,73.4333,Sindhiyapura
13,Kavieng,PG,82.65,broken clouds,-2.5744,150.7967,Nusa Island Retreat
16,Yinchuan,CN,81.45,few clouds,38.4681,106.2731,JJ Inns - Yinchuan New Moon Square Branch


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [ ]:
# In case of an empty DataFrame, load the sample data provided
# clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [39]:
# Set the file name.
output_data_file = "WeatherPy_vacation.csv"

# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [40]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [41]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd>Observatory Guest House</dd>\n<dt>City</dt><dd>Busselton</dd>\n<dt>Country</dt><dd>AU</dd>\n<dt>Current Weather</dt><dd>broken clouds and 70.11 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>عمارت ساعتی تفت</dd>\n<dt>City</dt><dd>Taft</dd>\n<dt>Country</dt><dd>IR</dd>\n<dt>Current Weather</dt><dd>clear sky and 78.75 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Motel 6 Sinton, TX</dd>\n<dt>City</dt><dd>San Patricio</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Current Weather</dt><dd>clear sky and 78.84 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Shandrani Beachcomber Resort & Spa</dd>\n<dt>City</dt><dd>Mahebourg</dd>\n<dt>Country</dt><dd>MU</dd>\n<dt>Current Weather</dt><dd>broken clouds and 76.24 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Best Western Plus Abbey Inn</dd>\n<dt>City</dt><dd>Saint George</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Current Weather</dt><dd>clear sky and 78.26 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt>

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [42]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
0,-33.6500,115.3333
1,31.7471,54.2088
2,28.0170,-97.5169
3,-20.4081,57.7000
4,37.1041,-113.5841
6,-20.7333,139.5000
10,-16.4833,-151.7500
12,22.1833,73.4333
13,-2.5744,150.7967
16,38.4681,106.2731


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [46]:
# Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Create a figure to add the Google map as a layer
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
# Display the figure containing the map
fig

Figure(layout=FigureLayout(height='420px'))